# Big table

Median and UI intervals for:

* Within diseases, proportions across regions

In [1]:
library(data.table)

In [2]:
sms <- list.files("/media/igna/Elements/HotelDieu/Cochrane/MappingRCTs_vs_Burden/Replicates/Metrics_over_repl/")

In [3]:
dis <- as.numeric(substr(sms,25,nchar(sms)-4))
dis

[1]  0  1 10 12 13 14 15 16 17 18 19  2 20 22 24 25 26  3  4  5  6  7  8

In [4]:
Mgbd <- read.table("../Data/27_gbd_groups.txt")

In [5]:
k <- 1
DF <- fread(paste(c("/media/igna/Elements/HotelDieu/Cochrane/MappingRCTs_vs_Burden/Replicates/Metrics_over_repl/Metrics_over_replicates_",
                    as.character(k),".txt"),collapse=""))
regs <- sort(unique(DF$Region))
regs <- regs[regs!="All"]

In [6]:
data_f <- data.frame()

for(k in dis[dis!=0]){

    DF <- fread(paste(c("/media/igna/Elements/HotelDieu/Cochrane/MappingRCTs_vs_Burden/Replicates/Metrics_over_repl/Metrics_over_replicates_",
                    as.character(k),".txt"),collapse=""))

    DFr <- DF[DF$Region%in%regs & DF$Dis == "dis",]
    DFr$RCTs_all <- rep(DF$RCTs[DF$Dis=="dis" & DF$Region=="All"],each=length(regs))
    DFr$RCTs_NHI <- rep(DF$RCTs[DF$Dis=="dis" & DF$Region=="Non-HI"],each=length(regs))
    DFr$Patients_all <- rep(DF$Patients[DF$Dis=="dis" & DF$Region=="All"],each=length(regs))
    DFr$Patients_NHI <- rep(DF$Patients[DF$Dis=="dis" & DF$Region=="Non-HI"],each=length(regs))

    df <- data.frame(cbind(regs,as.character(Mgbd$x[k]),
        do.call('rbind',by(DFr[DFr$RCTs_all!=0,],
                           DFr$Region[DFr$RCTs_all!=0],
                           function(x){100*quantile(x$RCTs/x$RCTs_all,probs=c(0.025,0.5,0.975))})),
        do.call('rbind',by(DFr[DFr$RCTs_all!=0,],
                           DFr$Region[DFr$RCTs_all!=0],
                           function(x){100*quantile(x$Patients/x$Patients_all,probs=c(0.025,0.5,0.975))})),
        do.call('rbind',by(DFr[DFr$RCTs_NHI!=0,],
                           DFr$Region[DFr$RCTs_NHI!=0],
                           function(x){100*quantile(x$RCTs/x$RCTs_NHI,probs=c(0.025,0.5,0.975))})),
        do.call('rbind',by(DFr[DFr$RCTs_NHI!=0,],
                           DFr$Region[DFr$RCTs_NHI!=0],
                           function(x){100*quantile(x$Patients/x$Patients_NHI,probs=c(0.025,0.5,0.975))})))
        )

    names(df) <- c("Region","Disease",
               paste(paste("Prop_all","RCTs",sep="_"),c("low","med","up"),sep="_"),
               paste(paste("Prop_all","Patients",sep="_"),c("low","med","up"),sep="_"),
               paste(paste("Prop_NHI","RCTs",sep="_"),c("low","med","up"),sep="_"),
               paste(paste("Prop_NHI","Patients",sep="_"),c("low","med","up"),sep="_"))

    data_f <- rbind(data_f,df)
}

In [7]:
rownames(data_f) <- NULL

In [8]:
data_f[data_f$Region%in%c("High-income","Non-HI"),grep("NHI",names(data_f))] <- NA

In [9]:
head(data_f)

,Region,Disease,Prop_all_RCTs_low,Prop_all_RCTs_med,Prop_all_RCTs_up,Prop_all_Patients_low,Prop_all_Patients_med,Prop_all_Patients_up,Prop_NHI_RCTs_low,Prop_NHI_RCTs_med,Prop_NHI_RCTs_up,Prop_NHI_Patients_low,Prop_NHI_Patients_med,Prop_NHI_Patients_up
1,"Central Europe, Eastern Europe, and Central Asia",Tuberculosis,3.69003740785565,5.80474934036939,8.13568226393947,0.0586688565808127,0.146246923590507,0.953885801111556,5.08474576271187,8.01526717557252,11.486262637802,0.0590392693376528,0.147773696961608,1.04143096553855
2,High-income,Tuberculosis,25.2298560242485,32.6086956521739,43.75,0.336494708967389,0.872099767046211,7.25688271653597,NA,NA,NA,NA,NA,NA
3,Latin America and Caribbean,Tuberculosis,6.90279038112523,9.6969696969697,12.5560538116592,0.150579073933349,1.28018189138977,7.01217162568154,9.86401964418087,13.3333333333333,17.0268295534253,0.152401364302237,1.29330112511701,7.5290188304808
4,Non-HI,Tuberculosis,62.0564190128775,72.9805013927577,80.0802811870901,92.7431172834641,99.1279002329537,99.6635052910327,NA,NA,NA,NA,NA,NA
5,North Africa and Middle East,Tuberculosis,3.59039767997637,5.86510263929619,8.48484848484849,0.0305870150014081,0.110434902108221,0.672088522128384,4.84384164222874,8.05369127516778,12.5,0.0307555608696088,0.111561248665482,0.721441277485596
6,South Asia,Tuberculosis,9.76133765243902,13.2315521628499,16.4971649484536,0.25035996641323,0.549502341176646,4.31949192113607,14.1666666666667,18.232044198895,22.2222222222222,0.252492059123606,0.555320687720408,4.66627459802023


In [10]:
write.table(data_f,"../Data/RCTs_and_Patients_prop_among_all_and_HI_median_UI_across_regions_per_disease.txt")